Load the __importnb__ to import notebooks.

[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/deathbeds/importnb/master?filepath=readme.ipynb)[![Build Status](https://travis-ci.org/deathbeds/importnb.svg?branch=master)](https://travis-ci.org/deathbeds/importnb)

## Jupyter Extension

In [1]:
    if __name__ == '__main__':
        %reload_ext importnb
        import readme
        assert readme.foo is 42
        assert readme.__file__.endswith('.ipynb')
    else: 
        foo = 42
        

Notebooks maybe reloaded with the standard Python Import machinery.

In [2]:
    if __name__ == '__main__':
        from importnb import Notebook, reload
        reload(readme)
        %unload_ext importnb

## Context Manager

In [3]:
    if __name__ == '__main__':
        try:  
            reload(readme)
            assert None, """Reloading should not work when the extension is unloaded"""
        except AttributeError: 
            with Notebook(): 
                reload(readme)

## Developer

In [5]:
    if __name__ == '__main__':
        !jupyter nbconvert --to markdown readme.ipynb
        !jupyter nbconvert --to rst readme.ipynb

[NbConvertApp] Converting notebook readme.ipynb to markdown
[NbConvertApp] Writing 1308 bytes to readme.md
[NbConvertApp] Converting notebook readme.ipynb to rst
[NbConvertApp] Writing 1536 bytes to readme.rst
